In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (1).csv


In [0]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
import timeit
import re

In [0]:
import io
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [5]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [0]:
import io
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [0]:

complete = pd.DataFrame(train.append(test))

complete = complete.drop('id', axis = 1)

In [8]:
cont_vars = complete.iloc[:,116:130].columns.values
print(len(cont_vars))
cat_vars = complete.iloc[:,:116].columns.values

14


In [0]:
complete.iloc[:,116:] = complete.iloc[:,116:].transform(lambda x: np.log1p(x))


In [0]:
complete = pd.get_dummies(complete.iloc[:,:])

In [11]:
train_new = complete.iloc[:188318,:]
print(train_new.shape)
test_new = complete.iloc[188318:,:]
test_new = test_new.drop('loss', axis = 1)
print(test_new.shape)

(188318, 1191)
(125546, 1190)


In [0]:
from sklearn.model_selection import train_test_split

In [13]:
y = train_new['loss']
y = y.reshape(train_new.shape[0], 1) # convert to nd numpy array
print(type(y))
X = train_new.drop(['loss'], axis = 1)
X= X.as_matrix() # convert to nd numpy array
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.1, random_state = 20)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


<class 'numpy.ndarray'>


In [0]:
from sklearn.metrics import mean_absolute_error as mae, r2_score as rs
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


In [0]:
lasso1 = Lasso(alpha = 0.001, random_state = 20)

In [16]:
lasso1.fit(X_train,y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=20,
   selection='cyclic', tol=0.0001, warm_start=False)

In [0]:
lasso_coeff = lasso1.coef_    #Finding the important coefficients.
index = []
for i,val in enumerate(lasso_coeff):
    if val!=0:
        index.append(i)

In [21]:
test_new.shape

(125546, 155)

In [19]:
X_train = X_train[:,index]                    #Selecting only important features in the train set
print(X_train.shape)

X_val = X_val[:,index]                     #Selecting only important features in the validation set
X_val.shape
test_new = test_new.iloc[:,index]
test_new.shape

IndexError: ignored

In [22]:
l1 = lasso1.fit(X_train, y_train)
result = mae(np.expm1(y_val), np.expm1(lasso1.predict(X_val)))          #Finding the mean absolute error
print(result)      #MAE = 1273.7017847938228


1273.7017847938225


In [0]:
from xgboost import XGBRegressor

In [0]:

param = {'n_estimators': [500],'max_depth': [6], 'learning_rate':  [0.1], 'subsample': [0.8]}
xgb_reg = XGBRegressor()
xgb_cv = GridSearchCV(xgb_reg, param, cv = 3, verbose = 3, n_jobs = 1)

In [27]:
xgb_cv.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] learning_rate=0.1, max_depth=6, n_estimators=500, subsample=0.8 .
[CV]  learning_rate=0.1, max_depth=6, n_estimators=500, subsample=0.8, score=0.5487093394878765, total= 6.5min
[CV] learning_rate=0.1, max_depth=6, n_estimators=500, subsample=0.8 .


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.6min remaining:    0.0s


[CV]  learning_rate=0.1, max_depth=6, n_estimators=500, subsample=0.8, score=0.5530543976542259, total= 6.5min
[CV] learning_rate=0.1, max_depth=6, n_estimators=500, subsample=0.8 .


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 13.1min remaining:    0.0s


[CV]  learning_rate=0.1, max_depth=6, n_estimators=500, subsample=0.8, score=0.559763454180012, total= 6.5min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 19.7min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [500], 'max_depth': [6], 'learning_rate': [0.1], 'subsample': [0.8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [28]:
print(xgb_cv.best_params_)
xgb_cv.best_score_

{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 500, 'subsample': 0.8}


0.5538423668213442

In [29]:
results = mae(np.expm1(y_val), np.expm1(xgb_cv.predict(X_val)))
print(results) 

1161.3300135687193


In [0]:
test_new = complete.iloc[188318:,:]

In [0]:
test_new = test_new.iloc[:,index].as_matrix()

In [0]:
ID = test.id

In [0]:
predictions = np.expm1(xgb_cv.predict(test_new))

In [0]:
X = np.concatenate((X_train,X_val),axis=0)

Y = np.concatenate((y_train,y_val),axis=0)

#Best model
model = XGBRegressor(n_estimators = 1000, max_depth=6, learning_rate =  0.1, subsample = 0.8 ,seed=0)
model.fit(X,Y)
predictions = np.expm1(model.predict(test_new))

NameError: ignored

In [0]:
predictions = np.expm1(model.predict(test_new))

In [0]:
X_train.shape

(169486, 155)

In [0]:
with open("out.csv", "w") as subfile:
    subfile.write("id,loss\n")
    for i, pred in enumerate(list(predictions)):
        subfile.write("%s,%s\n"%(ID[i],pred))

In [0]:
files.download("out.csv")